In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture('video3.mp4')

# Set up the initial frame as the background model
ret, frame = cap.read()
fgbg = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=False)
background = fgbg.apply(frame)

while True:
    # Read in the next frame and apply the background subtraction
    ret, frame = cap.read()
    if not ret:
        break
    fgmask = fgbg.apply(frame)

    # Apply some morphology to clean up the foreground mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    # Find the contours of the objects
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Show the numerical coordinates of the contours
    for contour in contours:
        # Get the centroid of the contour
        M = cv2.moments(contour)
        if M["m00"] == 0:
            continue
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])

        # Show the centroid coordinates on the image
        cv2.putText(frame, f"({cx}, {cy})", (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frames and the coordinate image
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()
